In [1]:
import os
import numpy as np
import pandas as pd
import torch
import random
import nltk

from tqdm import tqdm
from nltk.corpus import wordnet
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import f1_score, accuracy_score, classification_report

# Download necessary NLTK data (WordNet for synonyms)
nltk.download("wordnet")
nltk.download("omw-1.4")

# ----------------------------------------------------------------------------
#  1.1 Check GPU availability
# ----------------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Backe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Backe\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
#!unzip /kaggle/working/nltk_data/corpora/omw-1.4.zip -d /kaggle/working/nltk_data/corpora/
#!unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora/

In [2]:
# ----------------------------------------------------------------------------
#  2. LOAD YOUR DATA
#     Adjust these file paths to your environment. 
#     The CSVs must contain columns:
#       train.csv: claim, evidence, label
#       dev.csv:   claim, evidence, label
#       test.csv:  claim, evidence, (no label)
# ----------------------------------------------------------------------------

TRAIN_PATH = "data\\train.csv"
DEV_PATH   = "data\\dev.csv"
TEST_PATH  = "data\\test.csv"

BEST_MODEL_PATH = "data\\taskC\\best_deberta_model.pt"
OUTPUT_PATH = "data\\taskC\\predictions.csv"

train_df = pd.read_csv(TRAIN_PATH)
dev_df   = pd.read_csv(DEV_PATH)
test_df  = pd.read_csv(TEST_PATH)

train_df.rename(columns={"Claim": "claim", "Evidence": "evidence"}, inplace=True)
dev_df.rename(columns={"Claim": "claim", "Evidence": "evidence"}, inplace=True)
test_df.rename(columns={"Claim": "claim", "Evidence": "evidence"}, inplace=True)

print("Train samples:", len(train_df))
print("Dev samples:", len(dev_df))
print("Test samples:", len(test_df))

# If labels are strings, map them to integer {0,1} or {0,1,2,...}.
# For ED, assume 2 classes: 0 = not evidence, 1 = relevant evidence
train_df["label"] = train_df["label"].astype(int)
dev_df["label"]   = dev_df["label"].astype(int)

train_df.head(3)


Train samples: 21508
Dev samples: 5926
Test samples: 4688


,claim,evidence,label
0,We should introduce school vouchers,"Among the many educational reform efforts, suc...",0
1,We should legalize insider trading,The U.S. Securities and Exchange Commission wa...,0
2,We should subsidize investigative journalism,"The film won an Emmy Award (1980), George Polk...",0


In [3]:
# ----------------------------------------------------------------------------
#  3. (OPTIONAL) DATA AUGMENTATION (Synonym Replacement)
#      - We'll replace 1 random word in claim/evidence with a WordNet synonym
#      - For demonstration, there's a 15% chance per example to create an
#        augmented copy.
# 
# ----------------------------------------------------------------------------

def synonym_replacement(sentence, n=1):
    """
    Replace 'n' words in 'sentence' with synonyms from WordNet, if possible.
    """
    words = sentence.split()
    if len(words) < 2:
        return sentence

    indices_to_replace = random.sample(range(len(words)), k=min(n, len(words)))
    new_words = words[:]
    for i in indices_to_replace:
        word = words[i]
        syns = wordnet.synsets(word)
        if not syns:
            continue
        # For simplicity, pick from the first synset's lemmas
        lemmas = syns[0].lemma_names()
        # Filter out lemmas that are the same as the original
        lemmas = [l for l in lemmas if l.lower() != word.lower()]
        if len(lemmas) == 0:
            continue
        new_words[i] = random.choice(lemmas)
    return " ".join(new_words)


def augment_dataframe(df, alpha=0.15):
    """
    For each row, with probability alpha, create an augmented copy.
    Return a new DataFrame with both original and augmented samples.
    """
    augmented_rows = []
    for _, row in df.iterrows():
        # Original row
        augmented_rows.append(row.to_dict())
        
        if random.random() < alpha:
            new_row = row.copy()
            # Randomly augment claim or evidence
            if random.random() < 0.5:
                new_row["claim"] = synonym_replacement(row["claim"], n=1)
            else:
                new_row["evidence"] = synonym_replacement(row["evidence"], n=1)
            augmented_rows.append(new_row.to_dict())
    return pd.DataFrame(augmented_rows)

# Let's do a random seed for reproducibility
random.seed(42)

# AUGMENT the training set (remove if undesired)
augmented_train_df = augment_dataframe(train_df, alpha=0.15)
print("Original train size:", len(train_df), 
      " => After augmentation:", len(augmented_train_df))

train_df = augmented_train_df.reset_index(drop=True)


Original train size: 21508  => After augmentation: 24819


In [4]:
# ----------------------------------------------------------------------------
#  4. CREATE HUGGING FACE DATASETS
# ----------------------------------------------------------------------------

train_dataset = Dataset.from_pandas(train_df)
dev_dataset   = Dataset.from_pandas(dev_df)
test_dataset  = Dataset.from_pandas(test_df)

dataset_dict = DatasetDict({
    "train": train_dataset,
    "dev":   dev_dataset,
    "test":  test_dataset
})
dataset_dict


DatasetDict({
    train: Dataset({
        features: ['claim', 'evidence', 'label'],
        num_rows: 24819
    })
    dev: Dataset({
        features: ['claim', 'evidence', 'label'],
        num_rows: 5926
    })
    test: Dataset({
        features: ['claim', 'evidence'],
        num_rows: 4688
    })
})

In [5]:
# ----------------------------------------------------------------------------
#  5. TOKENIZATION
#     We use a powerful model: DeBERTa v3 (microsoft/deberta-v3-base)
#     which is known to outperform standard BERT on many tasks.
# ----------------------------------------------------------------------------

model_name = "microsoft/deberta-v3-base"

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples["claim"],
        examples["evidence"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

encoded_dataset = dataset_dict.map(tokenize_function, batched=True)


c:\Users\Backe\Documents\Manchester Uni\Year 3\NLU - Project\venv\lib\site-packages\transformers\convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/24819 [00:00<?, ? examples/s]

Map:   0%|          | 0/5926 [00:00<?, ? examples/s]

Map:   0%|          | 0/4688 [00:00<?, ? examples/s]

In [6]:
#Handling the labels in the dataset
encoded_dataset["train"] = encoded_dataset["train"].rename_column("label", "labels")
encoded_dataset["dev"]   = encoded_dataset["dev"].rename_column("label", "labels")

encoded_dataset["train"] = encoded_dataset["train"].remove_columns(["claim", "evidence"])
encoded_dataset["dev"]   = encoded_dataset["dev"].remove_columns(["claim", "evidence"])

encoded_dataset["test"]  = encoded_dataset["test"].remove_columns(["claim", "evidence"])

encoded_dataset["train"].set_format("torch")
encoded_dataset["dev"].set_format("torch")
encoded_dataset["test"].set_format("torch")

In [7]:
# ----------------------------------------------------------------------------
#  6. CUSTOM MODEL: Focal Loss or Label Smoothing
#     overridomg forward() to allow advanced loss functions.
# ----------------------------------------------------------------------------

import torch.nn as nn

class CustomDebertaModel(nn.Module):
    def __init__(self, model_name, num_labels=2, use_focal_loss=False, gamma=2.0, label_smoothing=0.0):
        super().__init__()
        self.num_labels = num_labels
        self.use_focal_loss = use_focal_loss
        self.gamma = gamma
        self.label_smoothing = label_smoothing
        
        # Load the pre-trained DeBERTa classification model
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        # DeBERTa forward pass (omit internal CE)
        kwargs.pop("num_items_in_batch", None)

        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=None,
            **kwargs
        )
        logits = outputs.logits  # shape: (batch_size, num_labels)
        
        loss = None
        if labels is not None:
            if self.use_focal_loss:
                loss = self.focal_loss(logits, labels, self.gamma)
            else:
                loss = self.label_smoothing_loss(logits, labels, self.label_smoothing)
        
        return {"loss": loss, "logits": logits}

    def focal_loss(self, logits, targets, gamma=2.0):
        # Focal Loss
        ce = nn.CrossEntropyLoss(reduction='none')(logits, targets)
        pt = torch.exp(-ce)
        focal = (1 - pt)**gamma * ce
        return focal.mean()

    def label_smoothing_loss(self, logits, targets, smoothing=0.0):
        if smoothing == 0.0:
            return nn.CrossEntropyLoss()(logits, targets)

        log_probs = nn.LogSoftmax(dim=-1)(logits)
        n_class = logits.size(1)
        with torch.no_grad():
            true_dist = torch.zeros_like(log_probs)
            true_dist.fill_(smoothing / (n_class - 1))
            true_dist.scatter_(1, targets.unsqueeze(1), 1.0 - smoothing)
        return torch.mean(torch.sum(-true_dist * log_probs, dim=1))


In [8]:
# ----------------------------------------------------------------------------
#  7. HYPERPARAMETER SEARCH WITH HYPEROPT
#     We'll define:
#       - learning_rate
#       - epochs
#       - batch_size
#       - use_focal_loss
#       - gamma (for focal loss)
#       - label_smoothing
# ----------------------------------------------------------------------------

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"f1": f1, "accuracy": acc}


def objective(space):
    """
    Hyperopt Objective:
      1) Create a CustomDebertaModel with the candidate hyperparams
      2) Train on train_dataset, evaluate on dev_dataset
      3) Return negative F1 (since Hyperopt minimizes)
    """
    learning_rate = space["learning_rate"]
    epochs        = int(space["epochs"])
    batch_size    = int(space["batch_size"])
    use_focal_loss = space["use_focal_loss"]
    gamma          = space["gamma"]
    label_smoothing = space["label_smoothing"]

    # Build the model
    model = CustomDebertaModel(
        model_name=model_name,
        num_labels=2,
        use_focal_loss=use_focal_loss,
        gamma=gamma,
        label_smoothing=label_smoothing
    )
    model.to(device)

    training_args = TrainingArguments(
        output_dir="./sota-ed-checkpoints",
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=learning_rate,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=8,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        save_total_limit=1,
        report_to="none", 
        logging_steps=1
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["dev"],
        processing_class=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    metrics = trainer.evaluate(encoded_dataset["dev"])
    f1 = metrics["eval_f1"]

    print(f"[Hyperopt] params={space} => F1={f1:.4f}")
    return {"loss": -f1, "status": STATUS_OK}


# Define search space
search_space = {
    "learning_rate":   hp.loguniform("learning_rate", np.log(1e-5), np.log(5e-4)),
    "epochs":          hp.choice("epochs", [2, 3, 4]),
    "batch_size":      hp.choice("batch_size", [4, 8, 16]),
    "use_focal_loss":  hp.choice("use_focal_loss", [False, True]),
    "gamma":           hp.quniform("gamma", 1.0, 5.0, 0.5),    
    "label_smoothing": hp.uniform("label_smoothing", 0.0, 0.2)
}

max_evals = 30 
trials = Trials()

best = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=max_evals,
    trials=trials
)

print("\nHyperopt best param indices:", best)


  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.016100,0.013316,0.887010,0.884576
2,0.009500,0.015070,0.888316,0.885926


[Hyperopt] params={'batch_size': 16, 'epochs': 2, 'gamma': 4.5, 'label_smoothing': 0.12159883445297202, 'learning_rate': 1.403352626309778e-05, 'use_focal_loss': True} => F1=0.8883
  3%|▎         | 1/30 [13:31<6:32:00, 811.05s/trial, best loss: -0.8883157289201107]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.151100,0.150067,0.607102,0.723253
2,0.149400,0.150219,0.607102,0.723253


[Hyperopt] params={'batch_size': 16, 'epochs': 2, 'gamma': 2.0, 'label_smoothing': 0.12099940322990729, 'learning_rate': 0.0004715680943388656, 'use_focal_loss': True} => F1=0.6071
 97%|█████████▋| 29/30 [14:27:38<18:39, 1119.60s/trial, best loss: -0.8927049346409967]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.012200,0.009392,0.885956,0.884914
2,0.006500,0.011801,0.882968,0.880695


[Hyperopt] params={'batch_size': 8, 'epochs': 2, 'gamma': 5.0, 'label_smoothing': 0.13984736905248035, 'learning_rate': 4.293097221060447e-05, 'use_focal_loss': True} => F1=0.8860
100%|██████████| 30/30 [14:45:26<00:00, 1770.89s/trial, best loss: -0.8927049346409967]

Hyperopt best param indices: {'batch_size': np.int64(2), 'epochs': np.int64(0), 'gamma': np.float64(4.0), 'label_smoothing': np.float64(0.1414757766530963), 'learning_rate': np.float64(4.798455916219888e-05), 'use_focal_loss': np.int64(1)}


In [9]:
# ----------------------------------------------------------------------------
#  7.1 Interpret best param indices from Hyperopt
# ----------------------------------------------------------------------------

epochs_options = [2, 3, 4]
batch_options  = [4, 8, 16]
use_focal_options = [False, True]

final_params = {
    "learning_rate":    best["learning_rate"],
    "epochs":           epochs_options[best["epochs"]],
    "batch_size":       batch_options[ best["batch_size"] ],
    "use_focal_loss":   use_focal_options[ best["use_focal_loss"] ],
    "gamma":            best["gamma"],
    "label_smoothing":  best["label_smoothing"]
}

print("Interpreted best hyperparams:\n", final_params)


Interpreted best hyperparams:
 {'learning_rate': np.float64(4.798455916219888e-05), 'epochs': 2, 'batch_size': 16, 'use_focal_loss': True, 'gamma': np.float64(4.0), 'label_smoothing': np.float64(0.1414757766530963)}


In [10]:
# ----------------------------------------------------------------------------
#  8. TRAIN A FINAL MODEL USING THE BEST HYPERPARAMS
# ----------------------------------------------------------------------------

best_model = CustomDebertaModel(
    model_name=model_name,
    num_labels=2,
    use_focal_loss=final_params["use_focal_loss"],
    gamma=final_params["gamma"],
    label_smoothing=final_params["label_smoothing"]
)
best_model.to(device)

training_args = TrainingArguments(
    output_dir="./final-sota-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=final_params["learning_rate"],
    num_train_epochs=final_params["epochs"],
    per_device_train_batch_size=final_params["batch_size"],
    per_device_eval_batch_size=8,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=1,
    report_to="none",  
    logging_steps=1
)

trainer = Trainer(
    model=best_model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["dev"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
results_dev = trainer.evaluate(encoded_dataset["dev"])
print("Final Dev Results:", results_dev)

# Classification report
preds_output = trainer.predict(encoded_dataset["dev"])
dev_preds = np.argmax(preds_output.predictions, axis=1)
dev_labels = preds_output.label_ids
print("\nDetailed Classification Report (Dev):")
print(classification_report(dev_labels, dev_preds, digits=4))


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Backe\AppData\Local\Temp\ipykernel_6292\4257955412.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.024700,0.019016,0.878786,0.875802
2,0.012700,0.020458,0.890355,0.888458


Final Dev Results: {'eval_loss': 0.02045840583741665, 'eval_f1': 0.8903553648924198, 'eval_accuracy': 0.8884576442794465, 'eval_runtime': 26.1667, 'eval_samples_per_second': 226.471, 'eval_steps_per_second': 28.318, 'epoch': 2.0}

Detailed Classification Report (Dev):
              precision    recall  f1-score   support

           0     0.9439    0.8992    0.9210      4286
           1     0.7656    0.8604    0.8102      1640

    accuracy                         0.8885      5926
   macro avg     0.8548    0.8798    0.8656      5926
weighted avg     0.8946    0.8885    0.8904      5926



In [16]:
# ----------------------------------------------------------------------------
# 8.1 SAVE THE BEST MODEL
# ----------------------------------------------------------------------------

hf_config_dict = trainer.model.model.config.to_dict()

# 2) Store everything in one dictionary
save_dict = {
    "model_state_dict": trainer.model.state_dict(),
    
    # The base Hugging Face model config 
    "hf_config": hf_config_dict,
    
    # Custom hyperparameters for re-initializing the custom wrapper
    "hyperparams": {
        "model_name": model_name,
        "num_labels": trainer.model.num_labels,
        "use_focal_loss": trainer.model.use_focal_loss,
        "gamma": float(trainer.model.gamma),
        "label_smoothing": float(trainer.model.label_smoothing)
    }
}

torch.save(save_dict, BEST_MODEL_PATH)
print(f"Model checkpoint saved to {BEST_MODEL_PATH}")

Model checkpoint saved to data\taskC\best_deberta_model.pt


In [14]:
# ----------------------------------------------------------------------------
# 9.1 INFERENCE ON THE DEV SET (codebench debugging)
# ----------------------------------------------------------------------------

from torch.utils.data import DataLoader

DEV_OUTPUT_PATH = "dev_predictions.csv"

# Create a DataLoader for the dev set
dev_loader = DataLoader(encoded_dataset["dev"], batch_size=8)
trainer.model.eval()
all_dev_preds = []

with torch.no_grad():
    for batch in dev_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = trainer.model(**batch)
        logits = outputs["logits"]
        preds = torch.argmax(logits, dim=1)
        all_dev_preds.extend(preds.cpu().tolist())

dev_pred_df = pd.DataFrame({"prediction": all_dev_preds})
dev_pred_df.to_csv(DEV_OUTPUT_PATH, index=False)
print(f"Dev predictions saved to {DEV_OUTPUT_PATH}")




Dev predictions saved to dev_predictions.csv


In [15]:
# ----------------------------------------------------------------------------
# 9.2 INFERENCE ON THE TEST SET 
# ----------------------------------------------------------------------------

from torch.utils.data import DataLoader

# Create a DataLoader for the test set
test_loader = DataLoader(encoded_dataset["test"], batch_size=8)
trainer.model.eval()
all_test_preds = []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = trainer.model(**batch)
        logits = outputs["logits"]
        preds = torch.argmax(logits, dim=1)
        all_test_preds.extend(preds.cpu().tolist())

test_pred_df = pd.DataFrame({"prediction": all_test_preds})
test_pred_df.to_csv(OUTPUT_PATH, index=False)
print(f"Test predictions saved to {OUTPUT_PATH}")

Test predictions saved to data\taskC\predictions.csv
